In [1]:
import os

# Subir un nivel respecto a la carpeta actual
os.chdir("..")

In [2]:
from Utils import load_parameters_file, from_matrix_to_preset, MSE, denormalize_preset, normalize_preset, pretty_print, PARAM_NAMES, from_preset_to_matrix
from Synth import Synth
import numpy as np
import cma
import matplotlib.pyplot as plt
from multiprocessing import Pool
from parallelEvaluation import evaluate_presets
from globals import SAMPLE_RATE, DURATION, PROCESSORS
from IPython.display import Audio

In [3]:
def render_presets(presets):
    synth = Synth(
        sample_rate=SAMPLE_RATE,
        duration=DURATION,
        presets=presets
    )
    return synth.process_audio()

In [4]:
target_audio = render_presets(load_parameters_file('target.json'))
Audio(target_audio[0], rate=SAMPLE_RATE)

In [ ]:
if __name__ == '__main__':
    target_mfcc = evaluate_presets(load_parameters_file('target.json'))
    # init_solution_mask = np.squeeze(from_preset_to_matrix(normalize_preset(load_parameters_file('init.json'))))

    optimizable_params = PARAM_NAMES

    x0 = np.random.rand(len(optimizable_params))
    sigma0 = 0.5

    # Crear estrategia
    es = cma.CMAEvolutionStrategy(x0, sigma0, {
        'popsize': 50, 
        'maxiter': 1000, 
        'bounds': [0, 1]
    })

    gen = 1

    with Pool(PROCESSORS) as pool:
        while not es.stop():
            # Generar población
            solutions = np.array(es.ask(), dtype=np.float32)  # devuelve una lista de individuos

            # solutions_mask = np.tile(init_solution_mask, (solutions.shape[0], 1)) # Para logica de optimizacion por partes
            # for i, param in enumerate(optimizable_params):
            #     idx = PARAM_NAMES.index(param)
            #     solutions_mask[:,idx] = solutions[:,i]

            solutions_splitted = np.split(solutions, PROCESSORS)
            presets_splitted = [denormalize_preset(from_matrix_to_preset(chunk)) for chunk in solutions_splitted]
            
            solutions_evaluated = pool.map(evaluate_presets, presets_splitted)
            solutions_evaluated = np.concatenate(solutions_evaluated)
            fitnesses = MSE(solutions_evaluated, target_mfcc)

            best_idx = np.argmin(fitnesses)          # índice del mejor fitness
            best_solution = solutions[best_idx]      # solución correspondiente
            best_fitness = fitnesses[best_idx]       # fitness correspondiente

            print("Gen", gen, "Mejor fitness:", best_fitness, "Sigma", es.sigma)

            es.tell(solutions, fitnesses)  # pasar fitness al algoritmo

            if best_fitness < 4.552335e-05:
                break

            gen += 1

    # Mejor solución
    best_solution = es.result.xbest
    print("Mejor individuo:", best_solution)

c:\Users\bryan\Documents\GitHub\Synth\venv\Lib\site-packages\cma\evolution_strategy.py:1531: UserWarning: Sampling standard deviation i=0 (and 41 others) at iteration 0 multiplied by 0.6666666666666666 to stds[0]=0.3333333333333333
  warnings.warn("Sampling standard deviation i={0}{4} at iteration {1}"


(25_w,50)-aCMA-ES (mu_w=14.0,w_1=14%) in dimension 42 (seed=427214, Tue Jan 13 21:58:00 2026)
Gen 1 Mejor fitness: 0.037755355 Sigma 0.5
Gen 2 Mejor fitness: 0.049432456 Sigma 0.4953699640719744
Gen 3 Mejor fitness: 0.038661752 Sigma 0.5038348385316975
Gen 4 Mejor fitness: 0.036111496 Sigma 0.5150560187765464
Gen 5 Mejor fitness: 0.03667913 Sigma 0.526191761374066
Gen 6 Mejor fitness: 0.03935709 Sigma 0.5289151968658242
Gen 7 Mejor fitness: 0.035765614 Sigma 0.5166257814471977
Gen 8 Mejor fitness: 0.036654998 Sigma 0.49482477874622116
Gen 9 Mejor fitness: 0.028469957 Sigma 0.484372574431467
Gen 10 Mejor fitness: 0.036441293 Sigma 0.46597088292534
Gen 11 Mejor fitness: 0.030606696 Sigma 0.4451892739061243
Gen 12 Mejor fitness: 0.0259809 Sigma 0.43038150780255563
Gen 13 Mejor fitness: 0.036815394 Sigma 0.41907297692974294
Gen 14 Mejor fitness: 0.03401508 Sigma 0.4035067738472307
Gen 15 Mejor fitness: 0.028953547 Sigma 0.39800798296299783
Gen 16 Mejor fitness: 0.03319208 Sigma 0.393855226

In [6]:
# solution = init_solution_mask
# for i, param in enumerate(optimizable_params):
#     idx = PARAM_NAMES.index(param)
#     solution[idx] = best_solution[i]

solution = np.expand_dims(best_solution, axis=0)
error = MSE(evaluate_presets(denormalize_preset(from_matrix_to_preset(solution))), target_mfcc)
print('Error', error)

target_audio = render_presets(load_parameters_file('target.json'))
predicted_audio = render_presets(denormalize_preset(from_matrix_to_preset(solution)))

Error [7.400768e-05]


In [ ]:
from scipy.io import wavfile

NAME = "exports/s3_a5"

Audio(target_audio[0], rate=SAMPLE_RATE)

audio_int16 = np.int16(target_audio[0] / np.max(np.abs(target_audio[0])) * 32767)
wavfile.write(f"{NAME}_target.wav", SAMPLE_RATE, audio_int16)


In [8]:
Audio(predicted_audio[0], rate=SAMPLE_RATE)

audio_int16 = np.int16(predicted_audio[0] / np.max(np.abs(predicted_audio[0])) * 32767)
wavfile.write(f"{NAME}_predicted_{error}.wav", SAMPLE_RATE, audio_int16)

In [9]:
denormalized_predicted_preset = denormalize_preset(from_matrix_to_preset(solution))
pretty_print(denormalized_predicted_preset)

{
  lfo1_rate:   [
    0.012355342958144472
  ]
  lfo1_shape:   [
    1.975404143333435
  ]
  lfo2_rate:   [
    0.02002104113352573
  ]
  lfo2_shape:   [
    0.506867527961731
  ]
  osc1_shape:   [
    0.00014873378677293658
  ]
  osc1_phase:   [
    0.052621543407440186
  ]
  osc1_volume:   [
    0.005017129238694906
  ]
  osc1_freq:   [
    21.59600157395256
  ]
  osc1_vdepth:   [
    0.9998621940612793
  ]
  osc1_pdepth:   [
    0.7255313992500305
  ]
  osc2_shape:   [
    3.004645586013794
  ]
  osc2_phase:   [
    9.240663530363236e-06
  ]
  osc2_volume:   [
    0.9973774552345276
  ]
  osc2_freq:   [
    440.13434823156535
  ]
  osc2_vdepth:   [
    0.016269220039248466
  ]
  osc2_pdepth:   [
    0.1685418337583542
  ]
  osc3_shape:   [
    0.17592185735702515
  ]
  osc3_phase:   [
    0.9639254808425903
  ]
  osc3_volume:   [
    0.6633896827697754
  ]
  osc3_freq:   [
    21184.502974797502
  ]
  osc3_vdepth:   [
    0.23246712982654572
  ]
  osc3_pdepth:   [
    0.78230810165

In [10]:
np.diagonal(es.C)

array([0.90758138, 1.37941657, 0.69013913, 0.91505754, 1.16280427,
       1.03132878, 0.06265334, 1.11912577, 0.90245794, 0.0016783 ,
       0.71713953, 0.83441928, 0.79575336, 1.05553582, 0.91794544,
       0.3967882 , 0.83999299, 1.03433737, 0.89223672, 0.90396204,
       1.23509273, 0.03235247, 0.83628901, 1.00398741, 0.96961722,
       1.08664865, 0.83914514, 0.29991495, 1.3872722 , 1.23373124,
       0.74124286, 0.06767763, 0.96081487, 0.66890379, 0.94607723,
       0.82733626, 1.13289711, 1.15824271, 1.02765284, 0.87709385,
       1.03655971, 0.9693604 ])